# Load data

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

def gmean(x):
    x = filter(lambda z: z == z, x)
    if len(x) == 0:
        return 0
    else:
        try:
            return np.prod(x)
        except:
            pass

In [2]:
rec_df = pd.read_csv('/Users/apple/Downloads/recs_df.csv')
del rec_df['Unnamed: 0']

In [3]:
#load clusters dataframe
cluster = pd.read_csv('../data_cleaning/cluster_most_reviewed.csv',index_col=0)

In [4]:
#load biz info
biz_info = pd.read_csv('../data_cleaning/biz_info_restaurant.csv',index_col=0)

In [5]:
items = np.random.choice(biz_info.business_id, size = 3)
biz_info[map(lambda x: x in items, biz_info['business_id'])].iloc[:,[2,3,1,4,5]]

,name_biz,stars_biz,count,address,neighborhood
99,El Dorado Cantina,4.5,177,3025 Sammy Davis Jr. Dr,NaN
926,Sammy's LA Pastrami & Burgers,3.5,51,2191 E Tropicana Ave,Southeast
1287,Fiesta Filipina Cuisine,3.5,36,"3310 S Jones Blvd, Ste A",Westside


In [6]:
clusters = list(cluster[map(lambda x: x in items, cluster.business_id)].cluster)
cluster_businesses = list(cluster[map(lambda x: x in clusters, cluster.cluster)].business_id)
cluster_recs = rec_df[map(lambda x, y: ((x in cluster_businesses) or (y in cluster_businesses)), rec_df.biz_1, rec_df.biz_2)]
cluster_recs['biz'] = map(lambda x, y: (x if y in items else y), cluster_recs.biz_1, cluster_recs.biz_2)
cluster_recs = list(biz_info[map(lambda x: x in list(cluster_recs.biz), biz_info.business_id)].sort(columns = 'count', ascending = False)['business_id'][0:10])

/Users/apple/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/apple/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [7]:
your_recs = rec_df[map(lambda x, y: ((x in items) or (y in items)), rec_df.biz_1, rec_df.biz_2)]
your_recs['biz'] = map(lambda x, y: (x if y in items else y), your_recs.biz_1, your_recs.biz_2)
del your_recs['biz_2']
your_recs =  your_recs.reset_index()
del your_recs['index']

/Users/apple/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [8]:
gmean_df = your_recs.groupby('biz').agg(gmean)
rec_list = []
rec_list = list(gmean_df[map(lambda x: x > 0 and x == x, gmean_df.at_cat_txt_us)].sort(columns = 'at_cat_txt_us', ascending = False)[0:7].index) + list(gmean_df[map(lambda x: x > 0 and x == x, gmean_df['3_rec'])].sort(columns = '3_rec', ascending = False)[0:(14-len(rec_list))].index)+ list((gmean_df[map(lambda x: x > 0 and x == x, gmean_df['2_rec'])].sort(columns = '2_rec', ascending = False)[0:(21-len(rec_list))].index))

/Users/apple/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [9]:
rec_list += cluster_recs
biz_info[map(lambda x: x in rec_list, biz_info['business_id'])].iloc[:,[2,3,1,4,5]]

,name_biz,stars_biz,count,address,neighborhood
0,Mon Ami Gabi,4.0,474,3655 Las Vegas Blvd S,The Strip
1,Bachi Burger,4.0,450,"470 E Windmill Ln, Ste 100",Southeast
3,Wicked Spoon,3.5,432,"The Cosmopolitan Of Las Vegas, 3708 Las Vegas ...",The Strip
4,Secret Pizza,4.0,421,"The Cosmopolitan Of Las Vegas, 3708 Las Vegas ...",The Strip
5,Monta Ramen,4.0,417,"5030 Spring Mountain Rd, Ste 6",Chinatown
6,Hash House A Go Go,4.0,416,6800 W Sahara Ave,Westside
8,Sushi House Goyemon,3.5,375,"5255 S Decatur Blvd, Ste118",NaN
9,Earl of Sandwich,4.5,372,3667 Las Vegas Blvd S,The Strip
10,Yard House Town Square,4.0,347,6593 Las Vegas Blvd S,Southeast
11,Pho Kim Long,3.5,344,4029 Spring Mountain Rd,Chinatown
